# TimesFM
 
TimesFM (Time Series Foundation Model) is a pretrained time-series foundation model developed by Google Research for time-series forecasting.

- It is launched a 500m checkpoint as a part of TimesFM-2.0 release. This new checkpoint can be upto 25% better than v1.0 on leading benchmarks and also has a 4 times longer max. context length.
- **It performs univariate time series forecasting** for context lengths up to 2048 timepoints and any horizon lengths, with an optional frequency indicator.
- It focuses on point forecasts. **We experimentally offer 10 quantile** heads but they have not been calibrated after pretraining.


TimesFM 2.0 has been added to GIFT-Eval which is one of the most comprehensive time-series bechmarks available. It takes the top spot in terms of aggregated MASE and CRPS, where it is 6% better than the next best model in terms of aggregated MASE.

### Usage
코드 설명

In [2]:
import timesfm

# Loading the timesfm-2.0 checkpoint:
# For PAX
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=24,          # 모델이 한꺼번에 예측해 내는 갯수
          num_layers=50,
          context_len=2048,        # 한 번에 볼 수 있는 과거 시계열의 최대 길이 = 2048

          use_positional_embedding=False,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-jax"),
  )

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Multiprocessing context has already been set.
Constructing model weights.


Constructed model weights in 6.20 seconds.
Restoring checkpoint from /home1/gkrtod35/.cache/huggingface/hub/models--google--timesfm-2.0-500m-jax/snapshots/47dedfcadf2abace1cc96071ddb798cfcd3bfcef/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 1.56 seconds.
Jitting decoding.
Jitted decoding in 34.90 seconds.


- **context_len**: It needs to be a multiplier of input_patch_len, i.e. a multiplier of 32.

실제로 데이터가 더 짧으면 **알아서** 패딩(padding)을, 더 길면 잘라내기(truncation) 를 해 주므로,
tfm.forecast()에 넘기는 시계열 길이는 임의로(1~2048 사이) 넣어도 상관없다.

- **horizon_len**: It generally recommend horizon length <= context length but it is not a requirement in the function call.

특별한 목적이 아니라면 예측할 시계열 길이가 학습할 길이보다 짧게 하자 


#### Perform inference
성능을 높이기 위해선 이 방법을 넣으라고 말한다

- 0 (default): 분, 시간, 일 단위처럼 하루 이하, 일단위까지 촘촘한 시계열
- 1: 주∙월 단위 (예: 주간 리포트, 월간 매출 등)
- 2: 분기∙연간 단위 (예: 분기별 매출, 연간 실적 등)

DataFrame을 쓸 때 방법은

0 로 매핑  →  T, MIN, H, D, B, U  
1 로 매핑  →  W, M  
2 로 매핑  →  Q, Y  

- ‘T’ = 분( 분 = T(=Time) ), ‘H’=시( Hour ), ‘D’=일( Day )
- ‘W’=주( Week ), ‘M’=월( Month )
- ‘Q’=분기( Quarter ), ‘Y’=연( Year )
- ‘B’(Business day), ‘U’(microsecond) 고빈도 옵션도 모두 0번 그룹

In [ ]:
import pandas as pd

# e.g. input_df is
#       unique_id  ds          y
# 0     T1         1975-12-31  697458.0
# 1     T1         1976-01-31  1187650.0
# 2     T1         1976-02-29  1069690.0
# 3     T1         1976-03-31  1078430.0
# 4     T1         1976-04-30  1059910.0
# ...   ...        ...         ...
# 8175  T99        1986-01-31  602.0
# 8176  T99        1986-02-28  684.0
# 8177  T99        1986-03-31  818.0
# 8178  T99        1986-04-30  836.0
# 8179  T99        1986-05-31  878.0

forecast_df = tfm.forecast_on_df(
    inputs=input_df,
    freq="M",          # monthly
    value_name="y",
    num_jobs=-1,
)

In [14]:
import pandas as pd

df = pd.read_excel('/home1/gkrtod35/file/solar_seoul.xlsx')
df['date'] = pd.to_datetime(df['date'])
df['datetime'] = df['date'] + pd.to_timedelta(df['time'], unit = 'h')

# 컬럼 이름 변경 및 unique_id 컬럼 추가
df_renamed = df[['datetime', 'solar generation']].rename(columns={'datetime':'ds'})
df_renamed['unique_id'] = 'solar_gen'

forecast_df = tfm.forecast_on_df(
    inputs=df_renamed,
    freq="h",  # hourly 데이터임을 명시
    value_name="solar generation",
    num_jobs=-1
)
forecast_df

Processing dataframe with multiple processes.


/home1/gkrtod35/miniconda3/envs/timesfm/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Finished preprocessing dataframe.
Finished forecasting.


,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
0,solar_gen,2023-12-01 00:00:00,-0.563668,-0.411823,-0.476037,-0.566645,-0.000924,-0.563668,-0.742101,-0.659100,-0.828420,-1.288279
1,solar_gen,2023-12-01 01:00:00,-0.895327,0.810916,0.099457,-0.067944,-0.114269,-0.895327,-0.808587,-0.954784,-0.967326,-1.406411
2,solar_gen,2023-12-01 02:00:00,-2.292804,-0.221481,-1.395725,-1.847097,-2.168489,-2.292804,-2.527087,-2.293997,-2.214677,-1.737606
3,solar_gen,2023-12-01 03:00:00,-0.845638,0.874767,0.338265,-0.184573,-0.163395,-0.845638,-0.625885,0.371830,0.608035,1.657946
4,solar_gen,2023-12-01 04:00:00,-0.571401,0.392318,-0.642709,-0.677616,-0.178875,-0.571401,-0.077131,0.907389,1.667334,2.598542
5,solar_gen,2023-12-01 05:00:00,1.015115,1.793175,0.791790,0.744863,1.292109,1.015115,1.382798,2.801320,3.163810,3.877274
6,solar_gen,2023-12-01 06:00:00,1.168942,0.886393,0.489936,0.196816,1.215567,1.168942,1.851352,3.288486,4.125086,5.293335
7,solar_gen,2023-12-01 07:00:00,1.417407,-0.655923,-0.257091,-0.117978,0.761750,1.417407,2.182743,3.318079,4.132752,5.079625
8,solar_gen,2023-12-01 08:00:00,0.557869,-0.535302,-0.804307,-0.862492,0.299661,0.557869,1.442743,2.716709,3.115819,3.853205
9,solar_gen,2023-12-01 09:00:00,0.135813,-2.028346,-2.056328,-1.777155,-0.688985,0.135813,1.142380,2.369926,3.314136,4.141083


#### 성능평가


In [19]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import numpy as np

rng = pd.date_range("2023-11-01", "2023-11-30 23:00", freq="H")
ts = pd.Series(np.sin(np.linspace(0, 10, len(rng))) + np.random.randn(len(rng))*0.1,
               index=rng)

# ───────────── 1) train/test 분리 ─────────────
train_end = datetime(2023, 11, 29, 23)  # inclusive
test_start = train_end + pd.Timedelta(hours=1)
test_end   = datetime(2023, 11, 30, 23)

train_ts = ts[:train_end]
test_ts  = ts[test_start:test_end]  

# ───────────── 2) TimesFM  ─────────────
tfm = timesfm.TimesFm(
    hparams=timesfm.TimesFmHparams(
        backend="gpu",            
        context_len=2048,         
        horizon_len=24,           
        per_core_batch_size=32,
        num_layers=8,             
        use_positional_embedding=False,
    ),
    checkpoint=timesfm.TimesFmCheckpoint(
        huggingface_repo_id="google/timesfm-2.0-500m-jax"
    ),
)

# ───────────── 3) 예측 수행 ─────────────
# TimesFM expects a list of numpy arrays. 길이 1짜리 리스트로 감싸기.
point_fc, _ = tfm.forecast(
    [train_ts.values],
    freq=[0],     # "H"→ 하루 이내 => freq=0
)

# point_fc.shape == (1, 24) 이므로 .flatten() 해서 1-D로 변환
pred = point_fc.flatten()

# ───────────── 4) 평가 지표 정의 ─────────────
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def mase(y_true, y_pred, y_ins, m=1):
    #
    naive_err = np.abs(y_ins[m:] - y_ins[:-m])
    scale = np.mean(naive_err)
    return mae(y_true, y_pred) / (scale + 1e-8)

# ───────────── 5) 결과 ─────────────
y_true = test_ts.values   
y_ins  = train_ts.values  #

print("RMSE :", rmse(y_true, pred))
print("MAE  :", mae(y_true, pred))
print("MASE :", mase(y_true, pred, y_ins, m=1))

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Multiprocessing context has already been set.
Constructing model weights.


Constructed model weights in 1.04 seconds.
Restoring checkpoint from /home1/gkrtod35/.cache/huggingface/hub/models--google--timesfm-2.0-500m-jax/snapshots/47dedfcadf2abace1cc96071ddb798cfcd3bfcef/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 1.09 seconds.
Jitting decoding.
Jitted decoding in 6.33 seconds.
RMSE : 0.4627080833181512
MAE  : 0.4423361520267723
MASE : 3.661345957945515


## TimesFM with Covariates

예측력을 높이는 방법으로써 제시되는 것이 외생변수(Exogenous Covariates)이다.

방법은 2가지인데

1. timesfm으로 구한 시계열의 잔차를 구해서 반응변수로 하고 외생변수를 설명변수로 해서 선형회귀를 만들고 예측한 잔차를 기존 예측에 더하는 방식
2. 시계열 데이터와 외생변수를 갖고 선형회귀를 만들고 그 잔차를 timesfm에 돌려서 최종예측을 한다.